### Deployement of the url models

In [5]:
PROJECT = 'qwiklabs-gcp-b27cb5778f18db6c' 
BUCKET = 'qwiklabs-gcp-b27cb5778f18db6c-2' 
REGION = 'europe-west1' 

In [6]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [7]:
import tensorflow as tf
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


### Train locally with gcloud

In [4]:
cd fraud-url

/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url


In [19]:
%bash
mkdir data
mv test.csv train.csv eval.csv data

In [6]:
%bash
DATADIR=$(pwd)/data
echo $DATADIR
OUTDIR=$(pwd)/trained/url_proj/
echo $OUTDIR
echo "${DATADIR}/train.csv"
echo $(pwd)/trainer.task
rm -rf $OUTDIR

gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=$(pwd)/trainer \
   -- \
   --TRAIN_DATA_PATH="${DATADIR}/train.csv" \
   --EVAL_DATA_PATH="${DATADIR}/eval.csv"  \
   --outdir=${OUTDIR} \
   --model="dnn"

/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/data
/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/trained/url_proj/
/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/data/train.csv
/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/trainer.task
bef help
1.8.0
/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/data/train.csv
Will train for 12000 steps using batch_size=256
Will use DNN size of [256, 128, 64, 32]


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--TRAIN_DATA_PATH=/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/data/train.csv', u'--EVAL_DATA_PATH=/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/data/eval.csv', u'--outdir=/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/fraud-url/trained/url_proj/', u'--model=dnn'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 10, '_tf_random_see

In [ ]:
!gsutil cp -r ./trained/url_proj/dnn gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/

### DEPLOYEMENT

In [6]:
%bash
gsutil ls gs://${BUCKET}/fraud-url/trained/url_proj/dnn/export/exporter/ | tail -10

gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618150/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618159/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618169/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618178/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618188/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618197/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618207/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618217/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618229/
gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618236/


In [8]:
%bash
MODEL_NAME="fraud"
MODEL_VERSION="dnn"
# MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/fraud-url/trained/url_proj/linear/export/exporter/ | tail -10)
MODEL_LOCATION=gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618150/
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
# gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
# gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying fraud dnn from gs://qwiklabs-gcp-b27cb5778f18db6c-2/fraud-url/trained/url_proj/dnn/export/exporter/1552618150/ ... this will take a few minutes


Creating version (this might take a few minutes)......
..........................................................................................................done.


In [22]:
pwd

'/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive'

In [ ]:
%bash
gsutil cp deployement_url.ipynbyement_ur

#### Online prediction

In [10]:
import ast
import numpy as np
try:
    from urllib.parse import urlparse
except ImportError:
     from urlparse import urlparse

In [11]:
num_int_features = ['url_length', 'contentLength', 'query_length', 'num_perc']
category_features = ['serverType', 'poweredBy', 'contentType']

In [12]:
from oauth2client.client import GoogleCredentials
import requests
import json

def request_prediction(example, model_version='linear', model_name='fraud'):
  
  token = GoogleCredentials.get_application_default().get_access_token().access_token
  api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
           .format(PROJECT, model_name, model_version)
  headers = {'Authorization': 'Bearer ' + token }
  data = {
    'instances': [example]
  }
  response = requests.post(api, json=data, headers=headers)
  return response

In [13]:
def process_response(response):
  pred_class = ast.literal_eval(response.content.decode())['predictions'][0]['classes'][0]
  proba_max = max(ast.literal_eval(response.content.decode())['predictions'][0]['probabilities'])
  proba_max = str(np.round(proba_max,2)*100)
  if pred_class=='1':
    result = 'TRUE : URL HIDDEN FRAUDULENT'
  else:
    result = 'FALSE : URL NOT HIDDEN FRAUDULENT BUT STILL BE CAREFUL'   
  output = result + " --- " "With probability : {} %".format(proba_max)
  print(output)

In [14]:
# FRAUDULENT EXAMPLE 
example_1 = {
  'url_length': 104,
  'contentLength': 4423,
  'query_length':  78,
  'num_perc': 0,
  'serverType': 'Microsoft-IIS/6.0',
  'poweredBy': 'ASP.NET',
  'contentType': 'text/html'
}

In [15]:
# NON FRAUDULENT EXAMPLE 
example_2 = {
  'url_length': 55,
  'contentLength': 0,
  'query_length':  13,
  'num_perc': 0,
  'serverType': 'Apache',
  'poweredBy': 'Unknown',
  'contentType': 'text/html; charset=utf-8'
}

### Predictions from the Linear Model

In [16]:
response_linear_1 = request_prediction(example_1, 'linear')
response_linear_2 = request_prediction(example_2, 'linear')

In [17]:
process_response(response_linear_1)
process_response(response_linear_2)

TRUE : URL HIDDEN FRAUDULENT --- With probability : 66.0 %
FALSE : URL NOT HIDDEN FRAUDULENT BUT STILL BE CAREFUL --- With probability : 98.0 %


### Predictions from the DNN Model

In [20]:
response_dnn_1 = request_prediction(example_1, 'dnn')
response_dnn_2 = request_prediction(example_2, 'dnn')

In [21]:
process_response(response_dnn_1)
process_response(response_dnn_2)

TRUE : URL HIDDEN FRAUDULENT --- With probability : 99.0 %
FALSE : URL NOT HIDDEN FRAUDULENT BUT STILL BE CAREFUL --- With probability : 100.0 %


### Predictions from the RNN Model (We Need)

### Predictions from the CNN Model (More Time)

In [85]:
import pandas as pd

test_df = pd.read_csv('./data/test.csv', sep=',')

def add_more_features(dataframe):
    df = dataframe.copy()

#     df['domain_temp'] = ""
#     df['domain_temp'] = df.url.apply(lambda row: extFind(row,True))
    df['url_length'] = df.url.apply(lambda row: len(row))
    df['query_string'] = df.url.apply(lambda row: urlparse(row).query if urlparse(row).query != '' else 'None')
    df['query_length'] = df.query_string.apply(lambda row: len(row))
#     df['primary'] = df.domain_temp.apply(lambda row: row[1] if row[0] in ['www','ww1','ww2'] else row[0])
#     df['primary_length'] = df.primary.apply(lambda row: len(row))
    df['num_periods'] = df.url.apply(lambda row: row.count("."))
    df['num_exclam'] = df.url.apply(lambda row: row.count("!"))
    df['num_quest'] = df.url.apply(lambda row: row.count("?"))
    df['num_perc'] = df.url.apply(lambda row: row.count("%"))
    df['num_numbers'] = df.url.apply(lambda row: sum(c.isdigit() for c in row))
#     df['last_year_modified'] = process_date(df)
#     df = df.drop(["query_string","primary","domain_temp"], axis=1)
    return df

test_df = add_more_features(test_df)

In [68]:
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [69]:
test_df.loc[0]

url                   http://www.fdp-muenchen-west.de/index.php?limi...
compromissionType                                            defacement
isHiddenFraudulent                                                False
contentLength                                                         0
serverType                                                       Apache
poweredBy                                                       Unknown
contentType                                    text/html; charset=utf-8
lastModified                              Tue, 29 Jan 2013 13:01:26 GMT
url_length                                                           55
query_string                                              limitstart=20
query_length                                                         13
num_periods                                                           3
num_exclam                                                            0
num_quest                                                       

#### Batch prediction using serving input function

In [ ]:
%writefile inputs.json
{
      'url_length': 104,
      'contentLength': 4423,
      'query_length':  78,
      'num_perc': 0,
      'serverType': 'Microsoft-IIS/6.0',
      'poweredBy': 'ASP.NET',
      'contentType': 'text/html'
}

In [ ]:
%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ml-engine jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=babyweight_sol --version=ml_on_gcp